In [1]:
# Libs 
import pandas   as pd
import requests as rq
from bs4 import BeautifulSoup

import os
from dotenv import load_dotenv

In [2]:
# Variables
url = "https://www.rmfmaxx.pl/muzyka/playlista"
header = {'User-Agent' : os.getenv('USER_AGENT')}

In [3]:
# Last 24h
extract = []
for hour in range(0,24):
    # Get Page
    relative_url = url + f"/{hour}#p"
    rmf_playlista = rq.get(relative_url, headers=header)
    soup = BeautifulSoup(rmf_playlista.content, 'html.parser')
    
    # Find music playlist
      # Select 'list-songs' first 'cause sidebar HopBec have the same div class 'row my-4 song-row'
    playlist = soup.find('div', class_ = 'list-songs')      
    playlist = playlist.find_all('div', class_ = 'row my-4 song-row')

    # Find all music played
    for music in playlist:
        # Artist
        music_artist = music.find('div', class_ = 'song-artists').get_text()
        # Title
        music_title = music.find('a', class_ = 'song-title text-muted').get_text()
        # Time [CET UTC+01:00 or CEST UTC+02:00]
        time_played = music.find('div', class_ = 'song-options small text-muted').get_text().strip()
        # Day 
        current_day = soup.find('b', class_='fw-500').get_text()
        
        extract.append({
            'Date Played': current_day,
            'Time Played': time_played,
            'Music Title': music_title,
            'Artist':  music_artist
        })

In [4]:
# Create Pandas Dataframe
playlista = pd.DataFrame(data=extract)

In [5]:
# Split for get Current Date and Artist Fuction
playlista['Date Played']  = playlista["Date Played"].str.split(',', n=2, expand=True)[1].str.split('(', n=2, expand=True)[0]
playlista[['Artist', 'Feat', 'Others_Artists']] = playlista["Artist"].str.split('/', n=2, expand=True).fillna('')

In [6]:
# Next commit add database
playlista.to_csv(path_or_buf='playlista.csv',index=False)